# NFT Market Analysis Project
## Part 1: OpenSea NFT Data Collection & Initial Cleaning
---

In [6]:
#Initial imports
import requests
import pandas as pd
import numpy as np
import datetime as dt
import os
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import json
import csv


#Imports required for pulling data using OpenSea API and cleaning the data
from helpers import parse_events_data, parse_assets_data, parse_sale_data, parse_listing_data
from pandas_profiling import ProfileReport
import pickle
from statistics import *
from scipy.stats import combine_pvalues
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import seaborn as sns
plt.style.use('ggplot')
import time
from datetime import date, timedelta, datetime
import glob


## Pull OpenSea data using API, save in Pandas dataframe, and clean the data

This section contains code pulled from Alex Duffy's Github which references base code by Adil Moujahid. \
Blog post located here: https://alxdfy.github.io/2021/09/19/data-mining-OpenSea_markdown.html#get-nft \
GitHub repository located here: https://github.com/AlxDfy/OpenSea_API_DataScience

In [7]:
"""Set a variable equal to the smart contract address for the desired NFT collection
Here we'll be evaluating the top 5 most popular (by volume) collections as of 11/16/2021: Bored Ape Yacht Club, Mutant Ape Yacht Club, CryptoPunks, The Sandbox, and Lil Baby Ape Club
Each collection's smart contract address is swapped in for the "asset_conctract_address variable to pull the data using the API and then saved as csv files. The smart contract addresses are as follows:
Bored Ape Yacht Club: 0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D
Mutant Ape Yacht Club: 0x60E4d786628Fea6478F785A6d7e704777c86a7c6
CryptoPunks: 0xb47e3cd837dDF8e4c57F05d70Ab865de6e193BBB
The Sandbox: 0x50f5474724e0Ee42D9a4e711ccFB275809Fd6d4a
Lil Baby Ape Club:0xB48eb7B72Ff5A4B5EF044EA9E706C990bb33884D
"""

asset_contract_address = "0xb47e3cd837dDF8e4c57F05d70Ab865de6e193BBB"

In [8]:
#Define function to pull NFT properties data from Opensea

def download_asset_info(save_location):
    if not os.path.isdir(save_location):
        os.makedirs(save_location)
    url = "https://api.opensea.io/api/v1/assets"
    listoassets = []

    for i in range(0, 3000):
        querystring = {"token_ids":list(range((i*30), (i*30)+30)),
                       "asset_contract_address":asset_contract_address,
                       "order_direction":"desc",
                       "offset":"0",
                       "limit":"30"}
        response = requests.request("GET", url, params=querystring)

        print(i, end=" ")
        if response.status_code != 200:
            print('error')
            print(response.json())
            break

        # Getting assets data
        assets = response.json()['assets']
        if assets == []:
            break
        # Parsing assets data
        parsed_assets = [parse_assets_data(asset) for asset in assets]
        # storing parsed events data into list
        listoassets.append(parsed_assets)
    
    # Flatten everything into one list
    listoassets = [item for sublist in listoassets for item in sublist]
    # Convert to df
    assets_df = pd.DataFrame(listoassets)

    #Save data in a new file
    with open(save_location + 'assets_df'+str(date.today())+r'.pkl', 'wb') as f:
        pickle.dump(assets_df, f)
        

In [9]:
#Define function to pull NFT sales data from Opensea
# Download sales info from start_date to end _date and save them all into their own day's files
# Default values from October 2020 to today which captures the majority of the sales

def download_sales_info(save_location, start_date = date(2020,10,1), end_date = date.today()):
    if not os.path.isdir(save_location):
        os.makedirs(save_location)
    url = "https://api.opensea.io/api/v1/events"
    # get the number of days that we want to download and save sales for
    delta = end_date - start_date
    count_days = int(delta.days)
    
    for i in range(count_days+1):
        sales_that_day = []
        # set start and end of the day we are checking, if it's today set end to current time
        if date.today() == (start_date + timedelta(days=i)):
            before = datetime.now()
            after = datetime.combine((start_date + timedelta(days=i)), datetime.min.time())
        else:
            before = datetime.combine((start_date + timedelta(days=i+1)), datetime.min.time())
            after = datetime.combine((start_date + timedelta(days=i)), datetime.min.time())
        # There are too many transactions, now have to break them up by chunks in the day
        hour_chunks = 24
        chunk_count = 24/hour_chunks
        time.sleep(.5)

        for chunk in range(int(chunk_count)):
            end = False
            for j in range(0, 35):
                time.sleep(.5)
                # add the hour_chunk to the start of the day (after) time for each chunk
                # use the actual before if we pass it chronologically though
                changed_before = after + timedelta(hours=hour_chunks*(chunk+1)) - timedelta(minutes = 1)
                changed_after = after + timedelta(hours = hour_chunks*(chunk))
                
                # this should only happen on the last chunk of a split day or if on current day
                if before < changed_before:
                    changed_before = before
                    end = True

                querystring = {"asset_contract_address":asset_contract_address,
                               "event_type":"successful",
                               "only_opensea":"true",
                               "offset":j*300,
                               "occurred_before":changed_before,
                               "occurred_after":changed_after,
                               "limit":"300"}
                headers = {"Accept": "application/json"}

                response = requests.request("GET", url, headers=headers, params=querystring)


                print(j, end=" ")
                if response.status_code != 200:
                    print('error')
                    print(response.json())
                    break

                #Getting assets sales data
                event_sales = response.json()['asset_events']

                if event_sales == []:
                    end =True
                    break

                # Parsing asset sales data
                parsed_event_sales = [parse_sale_data(sale) for sale in event_sales]
                # storing parsed events data into list
                sales_that_day.append(parsed_event_sales)
                # check if the last date in the list is the same day as 
                last_date = (datetime.strptime(parsed_event_sales[0]['timestamp'], '%Y-%m-%dT%H:%M:%S'))
                print(last_date)
            if end:
                break
        sales_that_day = [item for sublist in sales_that_day for item in sublist]
        
        print(str(len(sales_that_day))+ " sales saved to" + save_location + "events_sales_list_" + str(start_date + timedelta(days=i))+'.pkl')
        with open(save_location + "events_sales_list_" + str(start_date + timedelta(days=i))+'.pkl', 'wb') as f:
            pickle.dump(sales_that_day, f)

In [10]:
#Define function to pull NFT listings data from Opensea

# Download listings info from start_date to end _date and save them all into their own day's files
# Default values from October 2020 to today which captures the majority of the sales
def download_listings_info(save_location, start_date = date(2020,10,1), end_date = date.today()):
    if not os.path.isdir(save_location):
        os.makedirs(save_location)
    url = "https://api.opensea.io/api/v1/events"
    # get the number of days that we want to download and save listings for
    delta = end_date - start_date
    count_days = int(delta.days)
    
    for i in range(count_days+1):
        listings_that_day = []
        # set start and end of the day we are checking, if it's today set end to current time
        if date.today() == (start_date + timedelta(days=i)):
            before = datetime.now()
            after = datetime.combine((start_date + timedelta(days=i)), datetime.min.time())
        else:
            before = datetime.combine((start_date + timedelta(days=i+1)), datetime.min.time())
            after = datetime.combine((start_date + timedelta(days=i)), datetime.min.time())
        # There are too many transactions, now have to break them up by chunks in the day
        hour_chunks = 24
        chunk_count = 24/hour_chunks
        time.sleep(.5)

        
        for chunk in range(int(chunk_count)):
            end = False
            for j in range(0, 35):
                time.sleep(.5)
                # add the hour_chunk to the start of the day (after) time for each chunk
                # use the actual before if we pass it chronologically though
                changed_before = after + timedelta(hours=hour_chunks*(chunk+1)) - timedelta(minutes = 1)
                changed_after = after + timedelta(hours = hour_chunks*(chunk))
                
                # this should only happen on the last chunk of a split day or if on current day
                if before < changed_before:
                    changed_before = before
                    end = True

                querystring = {"asset_contract_address":asset_contract_address,
                               "event_type":"created",
                               "only_opensea":"true",
                               "offset":j*300,
                               "occurred_before":changed_before,
                               "occurred_after":changed_after,
                               "limit":"300"}
                headers = {"Accept": "application/json"}

                response = requests.request("GET", url, headers=headers, params=querystring)


                print(j, end=" ")
                if response.status_code != 200:
                    print('error')
                    print(response.json())
                    break

                #Getting assets listings data
                event_listings = response.json()['asset_events']

                if event_listings == []:
                    end = True
                    break

                # Parsing events listings data
                parsed_event_listings = [parse_listing_data(listing) for listing in event_listings]
                # storing parsed events data into list
                listings_that_day.append(parsed_event_listings)
                # check if the last date in the list is the same day as 
                print(parsed_event_listings[0]['created_date'])
            if end:
                break
        listings_that_day = [item for sublist in listings_that_day for item in sublist]
        
        print(str(len(listings_that_day))+ " listings saved to" + save_location +
              "events_listings_list_" + str(start_date + timedelta(days=i))+'.pkl')
        with open(save_location + "events_listings_list_" + str(start_date + timedelta(days=i))+'.pkl', 'wb') as f:
            pickle.dump(listings_that_day, f)

In [11]:
#Execute the pull requests for the properties and sales data for the NFTs

# Change location to suit your own needs
save_location = "./static/cryptopunks/"

# assets
download_asset_info(save_location)

# SALES
# download sales info from start_date to end_date e.g. date(2020,10,1), date.today() - timedelta(days=1), etc.
# from October 2020 to today which captures the majority of the sales

download_sales_info(save_location = save_location, start_date = date(2020,10,1))

# LISTINGS
# download listings info from start_date to end_date e.g. date(2020,10,1), date.today() - timedelta(days=1), etc.
# from October 2020 to today which captures the majority of the sales

download_listings_info(save_location = save_location, start_date = date(2020,10,1))

0 1 2 3 4 5 6 7 8 9 10 

KeyboardInterrupt: 

In [3]:
# load the sales lists, combine them, and turn into a DF
def load_sales_info(save_location):
    files = [filename for filename in os.listdir(save_location) if filename.startswith('events_sales')]
    all_sales = []
    # load all files for sales by day
    for file in files:
        with open(str(save_location) + str(file), 'rb') as f:
            all_sales.append(pickle.load(f))
    
    # flatten into one list
    all_sales = [item for sublist in all_sales for item in sublist]
    # convert to dataframe
    events_sales_df = pd.DataFrame(all_sales)
    
    return events_sales_df

# load the listing lists, combine them, and turn into a DF
def load_listings_info(save_location):
    files = [filename for filename in os.listdir(save_location) if filename.startswith('events_listing')]
    all_listings = []
    # load all files for listings by day
    for file in files:
        with open(str(save_location) + str(file), 'rb') as f:
            all_listings.append(pickle.load(f))
    
    # flatten into one list
    all_listings = [item for sublist in all_listings for item in sublist]
    # convert to dataframe
    events_listings_df = pd.DataFrame(all_listings)
    
    return events_listings_df

# load most recent saved assets df
def load_assets_info(save_location):
    files = glob.glob(str(save_location)+'assets_df????-??-??.pkl')
    with open(max(files, key=os.path.getctime), 'rb') as f:
        return pickle.load(f)

In [4]:
#load all our saved files

#SALES
events_sales_df = load_sales_info(save_location)
# Pre-processing
# Convert price from WEI to ETH & for now get rid of bundles and duplicates(?)
events_sales_df = events_sales_df[(events_sales_df['payment_token'] != 'USDC') & (events_sales_df['is_bundle'] == False)].copy()
events_sales_df = events_sales_df.loc[events_sales_df.astype(str).drop_duplicates().index]
events_sales_df['total_price'] = events_sales_df['total_price']/10.**18
# Change timestamp to datetime
events_sales_df['timestamp'] = pd.to_datetime(events_sales_df['timestamp'])
# Calculating the sale prices in USD
events_sales_df['total_price_usd'] = events_sales_df['total_price'] * events_sales_df['usd_price']


#LISTINGS
events_listings_df = load_listings_info(save_location)
# Pre-processing
# Convert price from WEI to ETH & for now get rid of bundles and duplicates(?)
events_listings_df = events_listings_df[(events_listings_df['payment_token'] != 'USDC') & (events_listings_df['is_bundle'] == False)].copy()
events_listings_df = events_listings_df.loc[events_listings_df.astype(str).drop_duplicates().index]
events_listings_df['starting_price'] = events_listings_df['starting_price']/10.**18
# Change timestamp to datetime
events_listings_df['created_date'] = pd.to_datetime(events_listings_df['created_date'])
# Calculating the sale prices in USD
events_listings_df['total_price_usd'] = events_listings_df['starting_price'] * events_listings_df['usd_price']

#ASSETS
assets_df = load_assets_info(save_location)

NameError: name 'save_location' is not defined

In [53]:
#Save the dataframes with the assets, sales, and listings as a csv files
assets_df.to_csv(path_or_buf='Output_csv_data_files/nfts/assets.csv')
events_sales_df.to_csv(path_or_buf='Output_csv_data_files/nfts/sales.csv')
events_listings_df.to_csv(path_or_buf='Output_csv_data_files/nfts/listings.csv')